# Interacting with SQL Databases Using Langchain's SQL Agents

## Libraries and Settings

In [ ]:
# Libraries
import os
import sqlite3
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage

# Load OpenAI API key from a top-level .env file (repo root)
def _find_repo_root(start: Path) -> Path:
    for parent in [start, *start.parents]:
        if (parent / 'requirements.txt').exists() or (parent / '.git').exists():
            return parent
    return start

_repo_root = _find_repo_root(Path.cwd())
load_dotenv(_repo_root / '.env')

api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError(
        "Missing OPENAI_API_KEY. Create a top-level .env file in the repository root "
        "and set OPENAI_API_KEY=..."
    )

# Path to the database
DB_PATH = './data/apartments_data.db'

# Settings
import warnings
warnings.filterwarnings("ignore")

# Current working directory
print(os.getcwd())

## Function to query the database and return the result as a DataFrame

In [ ]:
# Function to query the database and return the result as a DataFrame
def query_database(sql_query):
    """Execute an SQL query on the SQLite database and return the result as a DataFrame."""
    connection = sqlite3.connect(DB_PATH)
    cursor = connection.cursor()
    try:
        cursor.execute(sql_query)
        result = cursor.fetchall()
        columns = [description[0] for description in cursor.description]
        connection.close()
        df = pd.DataFrame(result, columns=columns)
        return df
    except Exception as e:
        connection.close()
        raise e

## Standard SQL queries

### Select all apartments

In [ ]:
# Count number of apartments
query = '''SELECT 
           address,
           rooms,
           area,
           price
           FROM apartments_table'''

# Query the database
result = query_database(query)

# Show the first 5 rows
result.head()

### Count the number of apartments

In [ ]:
# Count number of apartments
query = '''SELECT COUNT(*) AS apartment_count
           FROM apartments_table'''

# Query the database
result = query_database(query)

# Show the count
print("Number of apartments:", result['apartment_count'].iloc[0])

### Calculate the average price of selected apartments

In [ ]:
# Query
query = '''SELECT AVG(price) AS average_price
           FROM apartments_table
           WHERE rooms = 3.5;'''

# Query the database
result = query_database(query)

# Show the average price
print("Average price of apartments with 3.5 rooms and >= 100m² living area:", 
      f"{result['average_price'].iloc[0]:.4f}")

## Use an SQL Database Agent with LangChain to query the database

### LangChain Setup

In [ ]:
# Function to query the database and return the result as a DataFrame
def query_database(sql_query):
    "Execute an SQL query on the SQLite database and return the result as a DataFrame."
    connection = sqlite3.connect(DB_PATH)
    cursor = connection.cursor()
    try:
        cursor.execute(sql_query)
        result = cursor.fetchall()
        columns = [description[0] for description in cursor.description]
        connection.close()
        df = pd.DataFrame(result, columns=columns)
        return df, None
    except Exception as e:
        connection.close()
        return pd.DataFrame(), str(e)

# Function to generate an SQL query using LangChain
def langchain_sql_agent(user_prompt):
    """Generate and execute an SQL query using LangChain."""
    
    # Initialize the ChatOpenAI model
    chat = ChatOpenAI(
        temperature=0,
        model="gpt-3.5-turbo",
        openai_api_key=api_key
    )

    # System prompt for the agent
    system_prompt = SystemMessage(
        content="""You are an SQL expert. You translate natural language questions 
                    into valid SQL queries. Ensure the SQL query does not include 
                    any Markdown formatting.""")
    
    # User prompt
    user_prompt = HumanMessage(content=f"Translate this request into a valid SQL query: {user_prompt}")
    
    # Generate SQL query using LangChain
    try:
        response = chat([system_prompt, user_prompt])
        sql_query = response.content.strip()
        
        # Remove any Markdown formatting from the generated SQL query
        if sql_query.startswith("```sql") and sql_query.endswith("```"):
            sql_query = sql_query[6:-3].strip()
        
        print(f"Generated SQL Query:\n{sql_query}")
        
        # Execute the query and fetch results
        df, error = query_database(sql_query)

        if error:
            print(f"Error: {error}")
            return error
        else:
            return df
    except Exception as e:
        return str(e)

### Select all data in the database

In [ ]:
# Query
query = """Show colums address, rooms, area, price of all data in the apartments_table!"""

# Execute the agent
result = langchain_sql_agent(query)

# Show the result
result.head()

### Count the number of apartments

In [ ]:
# Query
query = """Show the number of apartments in the apartments_table!"""

# Execute the agent
langchain_sql_agent(query)

### Calculate the average price of selected apartments

In [ ]:
# Query
query = """What is the average price of apartments with 
           3.5 rooms in the apartments_table?"""

# Execute the agent
langchain_sql_agent(query)

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')